In [39]:
from langgraph.graph import StateGraph , START , END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict , Annotated
from pydantic import BaseModel , Field
import operator

In [40]:
load_dotenv()

True

In [41]:
model = ChatOpenAI(model='gpt-4o-mini')

Here we can make the schema (EvaluationSchema) for get the structured output from the model. feedback , score 

In [42]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed Feedback for teh essay")
    score: int =  Field(description="Score out of 10" , gt=0 , le=10)

In [43]:
structured_model = model.with_structured_output(EvaluationSchema)

`individual_scores: Annotated[list[int],operator.add]` In this we take a list data coming parallel from and take each input individually 

In [44]:
class EssayState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int],operator.add]
    avg_score: float

In [45]:
def evaluate_language(state:EssayState):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output =  structured_model.invoke(prompt)

    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}



In [46]:
def evaluate_analysis(state: EssayState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [47]:
def evaluate_thought(state: EssayState):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

In [48]:
def final_evaluation(state: EssayState):
    # Use safe access with get() to prevent KeyError
    language_fb = state.get("language_feedback", "")
    analysis_fb = state.get("analysis_feedback", "")
    clarity_fb = state.get("clarity_feedback", "")

    # Combine feedbacks
    prompt = (
        f"Based on the following feedbacks, create a summarized feedback:\n"
        f"Language feedback - {language_fb}\n"
        f"Depth of analysis feedback - {analysis_fb}\n"
        f"Clarity of thought feedback - {clarity_fb}"
    )

    overall_feedback = model.invoke(prompt).content

    # Handle missing or empty score list safely
    scores = state.get("individual_scores", [])
    avg_score = sum(scores) / len(scores) if scores else 0.0

    return {"overall_feedback": overall_feedback, "avg_score": avg_score}

In [49]:
graph = StateGraph(EssayState)

graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_thought", evaluate_thought)
graph.add_node("final_evaluation", final_evaluation)

# Start triggers all evaluations in parallel
graph.add_edge(START, "evaluate_language")
graph.add_edge(START, "evaluate_analysis")
graph.add_edge(START, "evaluate_thought")

# All evaluations feed into final summary
graph.add_edge("evaluate_language", "final_evaluation")
graph.add_edge("evaluate_analysis", "final_evaluation")
graph.add_edge("evaluate_thought", "final_evaluation")

graph.add_edge("final_evaluation", END)

workflow = graph.compile()

In [50]:
essay = """India and AI Time

Now world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.

India have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.

In farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.

But problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.

One more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.

India must all people together – govern, school, company and normal people. We teach AI and make sure AI not bad. Also talk to other country and learn from them.

If India use AI good way, we become strong, help poor and make better life. But if only rich use AI, and poor no get, then big bad thing happen.

So, in short, AI time in India have many hope and many danger. We must go right road. AI must help all people, not only some. Then India grow big and world say "good job India"."""

In [53]:
initial_state = {
    "essay": essay
}

workflow.invoke(initial_state)


{'essay': 'India and AI Time\n\nNow world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.\n\nIndia have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.\n\nIn farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.\n\nBut problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.\n\nOne more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.\n\n